In [1]:
# importing packages
import pandas as pd
import re
import os
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
# import ChromeDriverManager
from time import sleep
from time import time
start_time = time()
from selenium.webdriver.chrome.service import Service
from warnings import warn
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
# from fake_useragent import UserAgent

C:\Users\jparr\AppData\Local\Temp\ipykernel_3100\1217586740.py:7: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


In [2]:
# url here.
url = 'https://www.linkedin.com/jobs/search?keywords=%22Energy%20Sector%22%20OR%20Renewables%20OR%20%22hybrid%20vehicle%22%20OR%20%22electric%20mobility%22%20OR%20Energia%20OR%20oil&sortBy=DD&location=mexico&redirect=false&position=1&pageNum=0'
# url = 'https://www.linkedin.com/jobs/sortBy=DD&location=mexico&redirect=false&position=1&pageNum=0'


In [3]:
# options for Chromedriver - incognito browser, size, disabling infobars, extensions, etc.
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--incognito')
options.add_argument("start-maximized");
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")


In [4]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome("C:/Users/jparr/OneDrive - DAI/Mexico_FCO_Energy/chromedriver/chromedriver.exe", options = options)
driver.get(url)
sleep(7)
action = ActionChains(driver)
driver.set_window_size(1024, 600)
driver.maximize_window()

WebDriverException: Message: chrome not reachable


In [24]:
SCROLL_PAUSE_TIME = 3

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [38]:
count=15
while count>1:
    try:
        # ActionChains(driver).move_to_element(WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='btn btn-primary btn-lg']//span[@class='glyphicon glyphicon-play']")))).pause(3).click().perform()
        ActionChains(driver).move_to_element(WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label,'Load more results')]")))).pause(5).click().perform()
        print("LOAD MORE RESULTS button clicked")
        count = count - 1
    except TimeoutException:
        print("No more LOAD MORE RESULTS button to be clicked")
        break
print("No more LOAD MORE RESULTS button to be clicked")

LOAD MORE RESULTS button clicked
LOAD MORE RESULTS button clicked
LOAD MORE RESULTS button clicked
LOAD MORE RESULTS button clicked


In [45]:
# scroll down manually to get all relevant jobs

# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 51 jobs.


In [46]:
# post_title

In [47]:
# job_container   

In [49]:
# setting up list for job information
job_id = []
job_links = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    try:
        job_titles = job.find("h3", class_="base-search-card__title").text
    except:
        job_titles = "None"
    post_title.append(job_titles)
    
    # linkedin job id
    try:
        job_ids = job.find('a', href=True)['href']
        job_link = job_ids
        job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    except:
        job_ids = 'None'
        job_link = 'None'
    job_id.append(job_ids)
    job_links.append(job_link)
    
    
    # company name
    try:
        company_names = job.find('a', class_="hidden-nested-link").text
    except:
        company_names = 'None'
    company_name.append(company_names)
    
    # job location
    try:
        job_locations = job.find("span", class_="job-search-card__location").text
    except:
        job_locations = 'None'
    job_location.append(job_locations)
    
    # posting date
    try:
        post_dates = job.select_one('time')['datetime']
    except:
        post_dates = 'None'
    post_date.append(post_dates)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
   
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/div[1]/div/main/section[2]/ul/li[{}]'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    try:
        element1 = driver.find_element_by_xpath('/html/body/div[1]/div/section/div/section[2]/div[2]/section/button[1]')
        driver.execute_script("arguments[0].click();", element1)
    except:
        pass
    sleep(5)
    
    # job description
    try:
        # jobdesc_xpath = '/html/body/div[1]/div/section/div[2]/section[2]/div'
        # /html/body/div[1]/div/section/div[2]/div/section/div/div/section/div
        jobdesc_xpath = '/html/body/div[1]/div/section/div[2]/div/section/div/div/section/div'
        job_descs = driver.find_element_by_xpath(jobdesc_xpath).text.splitlines(0)[0]
    except:
        jobdesc_xpath = 'None'
        job_descs = 'None'
    job_desc.append(job_descs)
    
    # job criteria container below the description
    try:
        job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    except:
        job_criteria_container = 'None'
  #  all_job_criterias = job_criteria_container.find_all("span", class_='description__job-criteria-text description__job-criteria-text--criteria')
    
    # Seniority level
    try:
        seniority_xpath = '/html/body/div[1]/div/section/div[2]/div[1]/section[1]/div/ul/li[1]'
        seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    except:
        seniority_xpath = 'None'
        seniority = 'None'
    level.append(seniority)
    
    # Employment type
    try:
        type_xpath = '/html/body/div[1]/div/section/div[2]/div[1]/section[1]/div/ul/li[2]'
        employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    except:
        type_xpath = 'None'
        employment_type = 'None'
    emp_type.append(employment_type)
    
    # Job function
    try:
        function_xpath = '/html/body/div[1]/div/section/div[2]/div[1]/section[1]/div/ul/li[3]'
        job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    except:
        function_xpath = 'None'
        job_function = 'None'
    functions.append(job_function)
    
    # Industries
    try:
        industry_xpath = '/html/body/div[1]/div/section/div[2]/div[1]/section[1]/div/ul/li[4]'
        industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    except:
        industry_xpath = 'None'
        industry_type = 'None'
    industries.append(industry_type)
    x = x+1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/main/section[2]/ul/li[1]"}
  (Session info: chrome=101.0.4951.54)


In [13]:
# number for which information is not equal to None
print(len([n for n in job_id if n != 'None']))
print(len([n for n in job_links if n != 'None']))
print(len([n for n in post_date if n != 'None']))
print(len([n for n in company_name if n != 'None']))
print(len([n for n in post_title if n != 'None']))
print(len([n for n in job_location if n != 'None']))
print(len([n for n in job_desc if n != 'None']))
print(len([n for n in level if n != 'None']))

999
999
999
867
999
999
998
998


In [14]:
# to check if we have all information
print(len(job_id))
print(len(job_links))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

1999
1999
1999
1999
1999
1999
999
999
999
999
999


In [15]:
# creating a dataframe to take information in lists and store in common tidy structure
job_data_st = pd.DataFrame({'Job ID': job_links,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location
})


In [16]:
# creating a dataframe pt 2
job_data_add = pd.DataFrame({
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

In [17]:
job_data_st = job_data_st[job_data_st['Job ID'] != 'None']

In [18]:
# joining dframes
job_data = pd.concat([job_data_st.reset_index(drop=True), job_data_add], axis=1)


In [19]:
job_data

,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,https://mx.linkedin.com/jobs/view/ing-en-energ...,2022-01-20,\n \n ZenerSolar\n \n ...,\n \n Ing. en Energía recién...,"\n León, Guanajuato, Mexico\n ...",Por Crecimiento buscamos tu Talento INGENIERO ...,Full-time,None,None,None
1,https://mx.linkedin.com/jobs/view/gerente-de-g...,2022-02-03,\n \n Grupo Audace\n \n...,\n \n Gerente de gestión int...,"\n Cordoba, Mexico Metropolitan Are...",Escolaridad en ingeniería o administración afin,Full-time,None,None,None
2,https://mx.linkedin.com/jobs/view/especialista...,2022-02-03,\n \n AIEMAC A.C. \n \n...,\n \n Especialista en energí...,"\n Zacapu, Michoacán, Mexico\n ...",Vacante Disponible,Full-time,None,None,None
3,https://mx.linkedin.com/jobs/view/ingeniero-en...,2022-01-22,\n \n Natgas México\n \...,\n \n Ingeniero en Proyectos...,\n Queretaro Metropolitan Area\n ...,OBJETIVO DE LA POSICIÓN:,Full-time,None,None,None
4,https://mx.linkedin.com/jobs/view/ingeniero-pr...,2022-01-17,\n \n Volvo Group México\n ...,\n \n Ingeniero procesos JR\...,"\n Tultitlá de Mariano Escobedo, Mé...",Ingeniero de Procesos Jr,Associate,Full-time,Manufacturing and Engineering,Motor Vehicle Manufacturing
...,...,...,...,...,...,...,...,...,...,...
994,https://mx.linkedin.com/jobs/view/asesor-de-ve...,2022-01-25,None,\n \n Asesor de ventas\n ...,"\n San Luis Potosí, Mexico\n",Esta vacante viene de la bolsa de empleo Talen...,Entry level,Full-time,Sales,Retail Apparel and Fashion
995,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,\n \n Tutor with Preply\n ...,\n \n Profesor de Español (O...,"\n Tlalnepantla, México, Mexico\n ...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers
996,https://mx.linkedin.com/jobs/view/senior-java-...,2022-01-31,\n \n EPAM Anywhere\n \...,\n \n Senior Java Software E...,"\n Benito Juárez, Mexico City, Mexi...",We're looking for a remote Senior Java Softwar...,Mid-Senior level,Full-time,Engineering and Information Technology,"IT Services and IT Consulting, Software Develo..."
997,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,\n \n Tutor with Preply\n ...,\n \n Profesor de Español On...,"\n Atizapán de Zaragoza, México, Me...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers


In [20]:
# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')
job_data['Company Name'] = job_data['Company Name'].str.replace('\n',' ')
job_data['Post'] = job_data['Post'].str.replace('\n',' ')
job_data['Location'] = job_data['Location'].str.replace('\n',' ')
# print(job_data.info())
# job_data.head()

In [21]:
# save to excel
job_data.to_excel(r'C:\Users\jparr\OneDrive - DAI\Mexico_FCO_Energy\linkedin_df_july_29.xlsx')

In [22]:
job_data.head()

,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,https://mx.linkedin.com/jobs/view/ing-en-energ...,2022-01-20,ZenerSolar,Ing. en Energía recién e...,"León, Guanajuato, Mexico",Por Crecimiento buscamos tu Talento INGENIERO ...,Full-time,None,None,None
1,https://mx.linkedin.com/jobs/view/gerente-de-g...,2022-02-03,Grupo Audace,Gerente de gestión integ...,"Cordoba, Mexico Metropolitan Area...",Escolaridad en ingeniería o administración afin,Full-time,None,None,None
2,https://mx.linkedin.com/jobs/view/especialista...,2022-02-03,AIEMAC A.C.,Especialista en energía ...,"Zacapu, Michoacán, Mexico",Vacante Disponible,Full-time,None,None,None
3,https://mx.linkedin.com/jobs/view/ingeniero-en...,2022-01-22,Natgas México,Ingeniero en Proyectos d...,Queretaro Metropolitan Area ...,OBJETIVO DE LA POSICIÓN:,Full-time,None,None,None
4,https://mx.linkedin.com/jobs/view/ingeniero-pr...,2022-01-17,Volvo Group México ...,Ingeniero procesos JR ...,"Tultitlá de Mariano Escobedo, Méx...",Ingeniero de Procesos Jr,Associate,Full-time,Manufacturing and Engineering,Motor Vehicle Manufacturing


In [23]:
#screenshot driver
driver.save_screenshot('test.png')

True

In [24]:
# quit browse
driver.quit()

In [25]:
# job_data = pd.read_excel(r'C:\Users\jparr\OneDrive - DAI\Mexico_FCO_Energy\linkedin_df_july_23.xlsx')

In [26]:
# cleaning the data cont.
job_data[['Municipality', 'State', 'Country']] = job_data['Location'].str.split(pat = ",", n = 2, expand=True)

job_data = job_data.drop(columns = ['Country'])


job_data = job_data.rename(columns={"Job ID" : "Link", "Company Name" : "Company",
                        "Post": "Title", "Industry": "Sector"})

In [27]:
job_data

,Link,Date,Company,Title,Location,Description,Level,Type,Function,Sector,Municipality,State
0,https://mx.linkedin.com/jobs/view/ing-en-energ...,2022-01-20,ZenerSolar,Ing. en Energía recién e...,"León, Guanajuato, Mexico",Por Crecimiento buscamos tu Talento INGENIERO ...,Full-time,None,None,None,León,Guanajuato
1,https://mx.linkedin.com/jobs/view/gerente-de-g...,2022-02-03,Grupo Audace,Gerente de gestión integ...,"Cordoba, Mexico Metropolitan Area...",Escolaridad en ingeniería o administración afin,Full-time,None,None,None,Cordoba,Mexico Metropolitan Area
2,https://mx.linkedin.com/jobs/view/especialista...,2022-02-03,AIEMAC A.C.,Especialista en energía ...,"Zacapu, Michoacán, Mexico",Vacante Disponible,Full-time,None,None,None,Zacapu,Michoacán
3,https://mx.linkedin.com/jobs/view/ingeniero-en...,2022-01-22,Natgas México,Ingeniero en Proyectos d...,Queretaro Metropolitan Area ...,OBJETIVO DE LA POSICIÓN:,Full-time,None,None,None,Queretaro Metropolitan Area ...,None
4,https://mx.linkedin.com/jobs/view/ingeniero-pr...,2022-01-17,Volvo Group México ...,Ingeniero procesos JR ...,"Tultitlá de Mariano Escobedo, Méx...",Ingeniero de Procesos Jr,Associate,Full-time,Manufacturing and Engineering,Motor Vehicle Manufacturing,Tultitlá de Mariano Escobedo,México
...,...,...,...,...,...,...,...,...,...,...,...,...
994,https://mx.linkedin.com/jobs/view/asesor-de-ve...,2022-01-25,None,Asesor de ventas ...,"San Luis Potosí, Mexico",Esta vacante viene de la bolsa de empleo Talen...,Entry level,Full-time,Sales,Retail Apparel and Fashion,San Luis Potosí,Mexico
995,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,Tutor with Preply ...,Profesor de Español (Onl...,"Tlalnepantla, México, Mexico ...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers,Tlalnepantla,México
996,https://mx.linkedin.com/jobs/view/senior-java-...,2022-01-31,EPAM Anywhere,Senior Java Software Eng...,"Benito Juárez, Mexico City, Mexic...",We're looking for a remote Senior Java Softwar...,Mid-Senior level,Full-time,Engineering and Information Technology,"IT Services and IT Consulting, Software Develo...",Benito Juárez,Mexico City
997,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,Tutor with Preply ...,Profesor de Español Onli...,"Atizapán de Zaragoza, México, Mex...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers,Atizapán de Zaragoza,México


In [28]:
 
job_data.loc[job_data['State'].isnull() & job_data['Municipality'].notnull(), ['Municipality', 'State']] = job_data.loc[job_data['State'].isnull() & job_data['Municipality'].notnull(), ['State', 'Municipality']].values


In [29]:
# clean state and mun vars
job_data.State = job_data.State.str.lstrip()

job_data = job_data.replace({'State': {'Federal District' : 'Ciudad de México (Distrito Federal)',
                                                'Mexico City Metropolitan Area': 'Ciudad de México (Distrito Federal)',
                                               'México' : 'Estado de México', 
                                               'Mexico' : 'None'}})



In [30]:
job_data.loc[job_data['Municipality'] == 'Ciudad de México', 'State'] = 'Ciudad de México (Distrito Federal)'
job_data.loc[job_data['Municipality'] == 'Ciudad de México', 'Municipality'] = 'None'
job_data.loc[job_data['Municipality'] == 'Federal District', 'State'] = 'Ciudad de México (Distrito Federal)'
job_data.loc[job_data['Municipality'] == 'Federal District', 'Municipality'] = 'None'
job_data.loc[job_data['Municipality'] == 'México', 'State'] = 'Estado de México'
job_data.loc[job_data['Municipality'] == 'México', 'Municipality'] = 'None'
job_data.loc[job_data['Municipality'] == 'Mexico', 'Municipality'] = 'None'
job_data.loc[job_data['Municipality'] == 'Querétaro Centro', 'Municipality'] = 'Querétaro'

In [31]:
job_data.Municipality.unique()
job_data['Source'] = "LinkedIn"
# add in final columns - set to none to match data from other jobsites
job_data['Salary'] = 'None'
job_data['Synopsis'] = 'None'

In [32]:
job_data.State.unique()

array(['Guanajuato', 'Mexico Metropolitan Area           ', 'Michoacán',
       'Queretaro Metropolitan Area           ', 'Estado de México',
       'Querétaro', 'Campeche', 'Mexico           ', 'Tabasco',
       'Mexico City', 'Nuevo León', 'Jalisco',
       'Merida Metropolitan Area           ', 'Chihuahua', 'Puebla',
       'Mexico City Metropolitan Area           ', 'Tamaulipas',
       'Coahuila', 'Sonora', 'Zacatecas', 'Morelos', 'Baja California',
       'Yucatán', 'Aguascalientes',
       'Veracruz Metropolitan Area           ', 'San Luis Potosí',
       'Veracruz', 'Greater Coatzacoalcos Area           ',
       'Quintana Roo', 'Greater Pachuca de Soto Area           ',
       'Chiapas', 'Sinaloa', 'Monterrey Metropolitan Area           ',
       'Guerrero', 'Baja California Sur', 'Oaxaca',
       'Aguascalientes Metropolitan Area           ', 'Nayarit',
       'Tlaxcala', 'Durango', 'Hidalgo'], dtype=object)

In [33]:
job_data = job_data.replace({'State' : {'Ciudad de México' : 'Ciudad de México (Distrito Federal)',
                              'Mexico Metropolitan Area' : 'Ciudad de México (Distrito Federal)', 
                              'Monterrey Metropolitan Area' : 'Nuevo León', 
                              'Puebla' : 'Puebla (de los Angeles)', 
                              'México': 'None'}})

job_data = job_data.loc[job_data['State'].isin(['Nuevo León','Hidalgo','Chiapas',
                               'Jalisco','Durango','Ciudad de México (Distrito Federal)',
                               'Tlaxcala','Chihuahua','Baja California','Querétaro',
                               'San Luis Potosí','Guanajuato','Estado de México',
                               'Coahuila','Tabasco','Puebla (de los Angeles)',
                               'Morelos','Tamaulipas','Quintana Roo', 'Veracruz',
                               'Campeche','Aguascalientes','Nayarit','Michoacán',
                               'Colima','Sinaloa','Sonora','Yucatán','Oaxaca',
                               'Baja California Sur','Zacatecas','None']) == True]


In [34]:
# clean company

job_data['Company'] = job_data['Company'].str.strip()

# remove following
remove = ['Michael Page', 'Page Personnel', 'Job Fit', 'Working Mexico Headhunter', 'Page Personnel México',
        'Confidential', 'Confidencial', 'To view company name. See instructions below', 'Colocandote',
        'Working Mexico Headhunter', 'RemoteJobsWW', 'PageExecutive', 'Bumeran', 'Randstad', 'Adecco',
        'Cornerjobmx', 'Energyjobline', 'Computrabajo.Com', 'Confidencial', 'ListoPro',
        'Valle de Santiago', 'Texcoco', 'Benito Juárez', 'Monterrey', 'Coatzacoalcos', 'Silao', 'Mérida', 'San Martín de las Pirámides',
       'San Luis Potosí', 'Salvatierra', 'Miguel Hidalgo', 'Ecatepec de Morelos', 'Naucalpan de Juárez', 'Cuauhtémoc',
       'Veracruz', 'Salamanca', 'Juventino Rosas', 'Manzanillo', 'Oaxaca', 'Guadalajara', 'Puerto Vallarta', 'Iztapalapa', 'Guanajuato',
       'Minatitlán', 'Santa Cruz de Juventino Rosas', 'Tehuacán', 'Yautepec', 'Huixquilucan', 'Mexicali', 'Tlaquepaque', 'Morelia',
       'Guadalupe', 'Apodaca', 'Metepec', 'Lerma', 'Cuajimalpa de Morelos', 'Gustavo A. Madero', 'Toluca',
       'Tuxtla Gutiérrez', 'Zapopan', 'Atizapán de Zaragoza', 'Puebla',
       'Tlalnepantla de Baz', 'Gómez Palacio', 'Tlalpan', 'Cordoba', 'Nezahualcóyotl', 'Pachuca de Soto', 'Alvaro Obregón', 'Apizaco',
       'Lampazos de Naranjo', 'Tamazunchale', 'Santa Catarina',
       'Tlaxcala', 'Querétaro', 'Hermosillo', 'Chihuahua', 'Jiutepec', 'Paraíso', 'Torreon', 'Amatlán de los Reyes', 'Ciudad Juárez',
       'San Andrés Cholula', 'Azcapotzalco', 'Jocotitlán',
       'Aguascalientes', 'Roma', 'Zacoalco de Torres', 'Reynosa', 'Cuautitlán de García Barragán', 'Tijuana', 'Corregidora',
       'Cozumel', 'Isla Mujeres', 'Emiliano Zapata', 'Xalapa',
       'San Juan Del Rio', 'Zacatecas', 'Fresnillo', 'Los Cabos', 'Ensenada', 'Huejotzingo', 'Saltillo', 'Ixtlahuaca', 'Ahuazotepec', 'León', 'Tetla', 
       'Calera', 'Casas Grandes', 'Chalco',
       'Zapotlanejo', 'Ocoyoacac', 'Tepotzotlán', 'Otzolotepec', 'Irapuato', 'Río Bravo', 'San Martín Texmelucan',
       'Cuautitlán Izcalli', 'Macuspana', 'Temozón', 'Cárdenas', 'Teapa', 'San Juan Bautista Tuxtepec', 'Acayucan', 'Ciudad del Carmen',
       'Tecate', 'Tepic', 'Arandas', 'Cancún', 'El Espinal', 'Madero', 'Nuevo León','Hidalgo','Chiapas',
       'Jalisco','Durango','Ciudad de México (Distrito Federal)',
       'Tlaxcala','Chihuahua','Baja California','Querétaro','San Luis Potosí','Guanajuato','Estado de México',
       'Coahuila','Tabasco','Puebla (de los Angeles)','Morelos','Tamaulipas','Quintana Roo', 'Veracruz',
       'Campeche','Aguascalientes','Nayarit','Michoacán','Colima','Sinaloa','Sonora','Yucatán','Oaxaca',
       'Baja California Sur','Zacatecas']


# job_data.loc[job_data['Company'].isin(remove) == True, 'Company']  = "None"

In [35]:
job_data

,Link,Date,Company,Title,Location,Description,Level,Type,Function,Sector,Municipality,State,Source,Salary,Synopsis
0,https://mx.linkedin.com/jobs/view/ing-en-energ...,2022-01-20,ZenerSolar,Ing. en Energía recién e...,"León, Guanajuato, Mexico",Por Crecimiento buscamos tu Talento INGENIERO ...,Full-time,None,None,None,León,Guanajuato,LinkedIn,None,None
2,https://mx.linkedin.com/jobs/view/especialista...,2022-02-03,AIEMAC A.C.,Especialista en energía ...,"Zacapu, Michoacán, Mexico",Vacante Disponible,Full-time,None,None,None,Zacapu,Michoacán,LinkedIn,None,None
4,https://mx.linkedin.com/jobs/view/ingeniero-pr...,2022-01-17,Volvo Group México,Ingeniero procesos JR ...,"Tultitlá de Mariano Escobedo, Méx...",Ingeniero de Procesos Jr,Associate,Full-time,Manufacturing and Engineering,Motor Vehicle Manufacturing,Tultitlá de Mariano Escobedo,Estado de México,LinkedIn,None,None
5,https://mx.linkedin.com/jobs/view/ge-renewable...,2022-01-07,GE Renewable Energy,GE Renewable Energy Engi...,"Querétaro, Querétaro, Mexico ...",Job Description Summary If you love technology...,Entry level,Full-time,Other,"IT Services and IT Consulting, Appliances, Ele...",Querétaro,Querétaro,LinkedIn,None,None
6,https://mx.linkedin.com/jobs/view/managing-dir...,2022-01-21,Confidential,Managing Director ...,"Carmen, Campeche, Mexico",Requisitos:,Full-time,None,None,None,Carmen,Campeche,LinkedIn,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,Tutor with Preply,Profesor de Español Onli...,"Querétaro, Querétaro, Mexico ...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers,Querétaro,Querétaro,LinkedIn,None,None
990,https://mx.linkedin.com/jobs/view/promotor-a-n...,2022-01-09,None,Promotor/a Nogales ...,"Nogales, Durango, Mexico","Ubicación: Nogales, Sonora.",Entry level,Full-time,Marketing and Sales,IT Services and IT Consulting,Nogales,Durango,LinkedIn,None,None
991,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,Tutor with Preply,Profesor de Español (Onl...,"Atizapán de Zaragoza, México, Mex...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers,Atizapán de Zaragoza,Estado de México,LinkedIn,None,None
995,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,Tutor with Preply,Profesor de Español (Onl...,"Tlalnepantla, México, Mexico ...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers,Tlalnepantla,Estado de México,LinkedIn,None,None


In [36]:
job_data.to_excel(r'C:\Users\jparr\OneDrive - DAI\Mexico_FCO_Energy\linkedin_df_aug_7.xlsx')

In [37]:

# job_data = job_data.drop(columns = ['Unnamed: 0'])
job_data.columns= job_data.columns.str.strip().str.lower()


In [38]:
# remove all columns that are duplicates using link as ID
job_data.drop_duplicates(subset='link', keep="first")

,link,date,company,title,location,description,level,type,function,sector,municipality,state,source,salary,synopsis
0,https://mx.linkedin.com/jobs/view/ing-en-energ...,2022-01-20,ZenerSolar,Ing. en Energía recién e...,"León, Guanajuato, Mexico",Por Crecimiento buscamos tu Talento INGENIERO ...,Full-time,None,None,None,León,Guanajuato,LinkedIn,None,None
2,https://mx.linkedin.com/jobs/view/especialista...,2022-02-03,AIEMAC A.C.,Especialista en energía ...,"Zacapu, Michoacán, Mexico",Vacante Disponible,Full-time,None,None,None,Zacapu,Michoacán,LinkedIn,None,None
4,https://mx.linkedin.com/jobs/view/ingeniero-pr...,2022-01-17,Volvo Group México,Ingeniero procesos JR ...,"Tultitlá de Mariano Escobedo, Méx...",Ingeniero de Procesos Jr,Associate,Full-time,Manufacturing and Engineering,Motor Vehicle Manufacturing,Tultitlá de Mariano Escobedo,Estado de México,LinkedIn,None,None
5,https://mx.linkedin.com/jobs/view/ge-renewable...,2022-01-07,GE Renewable Energy,GE Renewable Energy Engi...,"Querétaro, Querétaro, Mexico ...",Job Description Summary If you love technology...,Entry level,Full-time,Other,"IT Services and IT Consulting, Appliances, Ele...",Querétaro,Querétaro,LinkedIn,None,None
6,https://mx.linkedin.com/jobs/view/managing-dir...,2022-01-21,Confidential,Managing Director ...,"Carmen, Campeche, Mexico",Requisitos:,Full-time,None,None,None,Carmen,Campeche,LinkedIn,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,Tutor with Preply,Profesor de Español Onli...,"Querétaro, Querétaro, Mexico ...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers,Querétaro,Querétaro,LinkedIn,None,None
990,https://mx.linkedin.com/jobs/view/promotor-a-n...,2022-01-09,None,Promotor/a Nogales ...,"Nogales, Durango, Mexico","Ubicación: Nogales, Sonora.",Entry level,Full-time,Marketing and Sales,IT Services and IT Consulting,Nogales,Durango,LinkedIn,None,None
991,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,Tutor with Preply,Profesor de Español (Onl...,"Atizapán de Zaragoza, México, Mex...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers,Atizapán de Zaragoza,Estado de México,LinkedIn,None,None
995,https://mx.linkedin.com/jobs/view/profesor-de-...,2022-02-01,Tutor with Preply,Profesor de Español (Onl...,"Tlalnepantla, México, Mexico ...",Estamos buscando profesores de Español online ...,Entry level,Full-time,Education and Training,E-Learning Providers,Tlalnepantla,Estado de México,LinkedIn,None,None


In [39]:
# connect postgresql database

import psycopg2
import psycopg2.extras as extras
import os
from io import StringIO
import sqlalchemy


In [40]:

# Here you want to change your database, username & password according to your own values
param_dic = {
    "host"      : "mexico-fco-db.cmjm2q5ghgua.us-east-1.rds.amazonaws.com",
    "database"  : "postgres",
    "user"      : "mexicoFCO",
    "password"  : "HC2g8Q7^taOq"
}


In [41]:

connection = psycopg2.connect(host = "mexico-fco-db.cmjm2q5ghgua.us-east-1.rds.amazonaws.com",
                 database = "postgres",
                 user = "mexicoFCO",
                 password = "HC2g8Q7^taOq")

cursor = connection.cursor()



In [42]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [43]:
# initial bulk upload with no prior records
# def execute_many(conn, df, table):
#     """
#     Using cursor.executemany() to insert the dataframe
#     """
#     # Create a list of tupples from the dataframe values
#     tuples = [tuple(x) for x in df.to_numpy()]
#     # Comma-separated dataframe columns
#     cols = ','.join(list(df.columns))
#     # SQL quert to execute
#     query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s, %%s,%%s,%%s, %%s,%%s,%%s, %%s,%%s,%%s, %%s,%%s,%%s)" % (table, cols)
#     cursor = conn.cursor()
#     try:
#         cursor.executemany(query, tuples)
#         conn.commit()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         conn.rollback()
#         cursor.close()
#         return 1
#     print("execute_many() done")
#     cursor.close()



In [44]:
# new and updated only to upload records not existing in current database
def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s, %%s,%%s,%%s, %%s,%%s,%%s, %%s,%%s,%%s, %%s,%%s,%%s) ON CONFLICT (link) DO UPDATE SET link = excluded.link;" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()
    


In [45]:
execute_many(conn = conn, df = job_data, table = 'jobs_dbase')

execute_many() done
